# Statistical tests

In [1]:
res_ml <- read.table("./smo_disc_pfi_lung_ml_100_iter_rep_kfold.csv", header = TRUE, sep = ",")

In [2]:
res_cnn <- read.table("./ros_pt_ft_disc_pfi_lung_ft_100_iter_rep_kfold.csv", header = TRUE, sep = ",")

In [3]:
res_cnn_mean <- read.table("./ros_mean_disc_pfi_lung_ft_100_iter_rep_kfold.csv", header = TRUE, sep = ",")

In [4]:
res_cnn_random <- read.table("./ros_random_disc_pfi_lung_ft_100_iter_rep_kfold.csv", header = TRUE, sep = ",")

In [5]:
res_mlnn <- read.table("./ros_mlnn_disc_pfi_lung_ft_100_iter_rep_kfold.csv", header = TRUE, sep = ",")

In [6]:
res_all <- cbind(res_ml, res_cnn, res_cnn_mean, res_cnn_random, res_mlnn)
summary(res_all)

   ANOVA.ANN         ANOVA.LR         ANOVA.RF        ANOVA.SVM     
 Min.   :0.5440   Min.   :0.5379   Min.   :0.5435   Min.   :0.5762  
 1st Qu.:0.6227   1st Qu.:0.6572   1st Qu.:0.6587   1st Qu.:0.6714  
 Median :0.6629   Median :0.7057   Median :0.6972   Median :0.7016  
 Mean   :0.6730   Mean   :0.6958   Mean   :0.6895   Mean   :0.7035  
 3rd Qu.:0.7193   3rd Qu.:0.7348   3rd Qu.:0.7288   3rd Qu.:0.7410  
 Max.   :0.8246   Max.   :0.8157   Max.   :0.8214   Max.   :0.8694  
    KPCA.ANN         KPCA.LR          KPCA.RF          KPCA.SVM     
 Min.   :0.5597   Min.   :0.5601   Min.   :0.5250   Min.   :0.5290  
 1st Qu.:0.6156   1st Qu.:0.6425   1st Qu.:0.6189   1st Qu.:0.6536  
 Median :0.6603   Median :0.6863   Median :0.6740   Median :0.6889  
 Mean   :0.6596   Mean   :0.6788   Mean   :0.6650   Mean   :0.7007  
 3rd Qu.:0.6932   3rd Qu.:0.7145   3rd Qu.:0.7118   3rd Qu.:0.7426  
 Max.   :0.7730   Max.   :0.7899   Max.   :0.7800   Max.   :0.8641  
    PCA.ANN           PCA.LR      

In [7]:
res_all_mean <- apply(res_all, 2, mean)
(res_all_mean <- res_all_mean[order(res_all_mean, decreasing = FALSE)])

PCA.ANN    PCA.RF  KPCA.ANN   KPCA.RF ANOVA.ANN   PCA.SVM   KPCA.LR    PCA.LR 
0.6474478 0.6498049 0.6595976 0.6649838 0.6730201 0.6736045 0.6787791 0.6808124 
 ANOVA.RF  ANOVA.LR    Random      Mean  KPCA.SVM ANOVA.SVM      MLNN  Fine.all 
0.6894997 0.6957767 0.6995690 0.6997772 0.7007381 0.7034664 0.7093653 0.7326019

In [8]:
fact <- c()
perf <- c()
for (n in names(res_all_mean)) {
    fact <- c(fact, rep(n, times = nrow(res_all)))
    perf <- c(perf, res_all[, n])
}

df_res_all <- data.frame(model = factor(fact, levels = names(res_all_mean)), result = perf)

In [9]:
head(df_res_all)

model,result
<fct>,<dbl>
PCA.ANN,0.7084677
PCA.ANN,0.5891129
PCA.ANN,0.7350806
PCA.ANN,0.6858871
PCA.ANN,0.6283422
PCA.ANN,0.7302419


## Adjust p-value
We adjust all computed p-values together.

In [10]:
compute_p_value <- function(main_model, sub_model, test_function = wilcox.test) {
    p_value <- c()
    for (s in sub_model) {
        p_value <- c(p_value, test_function(x = df_res_all[df_res_all[, "model"]==main_model, "result"], 
                                       y = df_res_all[df_res_all[, "model"]==s, "result"], 
                                       alternative = "greater", paired = TRUE)$p.value)
    }
    return(p_value)
}

In [11]:
colnames(res_all)

[1] "ANOVA.ANN" "ANOVA.LR"  "ANOVA.RF"  "ANOVA.SVM" "KPCA.ANN"  "KPCA.LR"  
 [7] "KPCA.RF"   "KPCA.SVM"  "PCA.ANN"   "PCA.LR"    "PCA.RF"    "PCA.SVM"  
[13] "Fine.all"  "Mean"      "Random"    "MLNN"

In [12]:
length(colnames(res_all))

[1] 16

In [13]:
# Compute CNN p-values
cnn_main_mod <- "Fine.all"
cnn_sub_mod <- colnames(res_all)[-13]
cnn_p_value <- compute_p_value(main_model = cnn_main_mod, sub_model = cnn_sub_mod, 
                                test_function = wilcox.test)
names(cnn_p_value) <- cnn_sub_mod

# Compute Deep-MLNN p-values
mlnn_main_mod <- "MLNN"
mlnn_sub_mod <- colnames(res_all)[c(1, 5, 9)]
mlnn_p_value <- compute_p_value(main_model = mlnn_main_mod, sub_model = mlnn_sub_mod, 
                                test_function = wilcox.test)
names(mlnn_p_value) <- mlnn_sub_mod

all_p_value <- c(cnn_p_value, mlnn_p_value)

# Adjust all computed p-values
bonf_adj_p_value <- p.adjust(p = all_p_value, method = "bonferroni")
holm_adj_p_value <- p.adjust(p = all_p_value, method = "holm")
hoch_adj_p_value <- p.adjust(p = all_p_value, method = "hochberg")

all_adj_p_value <- data.frame(Bonferroni = bonf_adj_p_value, Holm = holm_adj_p_value, Hochberg = hoch_adj_p_value)

In [14]:
# CNN vs all
cnn_vs_all <- all_adj_p_value[c(1:15), ]
rownames(cnn_vs_all) <- names(cnn_p_value)
cnn_vs_all

,Bonferroni,Holm,Hochberg
,<dbl>,<dbl>,<dbl>
ANOVA.ANN,7.004987e-07,4.280825e-07,4.280825e-07
ANOVA.LR,4.904557e-05,1.907328e-05,1.907328e-05
ANOVA.RF,7.294374e-06,3.241944e-06,3.241944e-06
ANOVA.SVM,1.034978e-03,2.874939e-04,2.874939e-04
KPCA.ANN,7.449362e-09,7.449362e-09,7.449362e-09
KPCA.LR,1.552697e-07,1.207653e-07,1.207653e-07
KPCA.RF,2.561227e-07,1.849775e-07,1.849775e-07
KPCA.SVM,8.362153e-03,9.291282e-04,4.646922e-04
PCA.ANN,1.137564e-08,1.074366e-08,1.074366e-08


In [15]:
# MLNN vs NN
deep_vs_ann <- all_adj_p_value[c(16:nrow(all_adj_p_value)), ]
rownames(deep_vs_ann) <- names(mlnn_p_value)
deep_vs_ann

,Bonferroni,Holm,Hochberg
,<dbl>,<dbl>,<dbl>
ANOVA.ANN,3.975193e-03,8.833763e-04,4.646922e-04
KPCA.ANN,6.442556e-06,3.221278e-06,3.221278e-06
PCA.ANN,5.500918e-07,3.667279e-07,3.667279e-07
